In [1]:
#Carrega as bibliotecas
import pandas as pd 
import numpy as np

pd.set_option('display.max_columns', None)

O procedimento "match" a seguir é usado para tomar valores de DataFrames com informações da escola com base nas entradas.
    match(x,df,busca,retorna)
    x: Valor que será buscado.
    df: DataFrame onde a busca será feita.
    busca: Coluna de df que será feita a busca
    retorna: Coluna com o valor que será retornado caso haja correspondência entre x e algum item do DataFrame 'df' na coluna 'busca.
Caso não haja correspondência entre x e qualquer valor na coluna de busca do DataFrame o procedimento retorna 'Escola não encontrada"

In [2]:
#Procedimento que retorna valor de um dataframe correspondente a um valor de busca.

def match(x,df,busca,retorna):
    if df[busca].isin([x]).any():
        return df[retorna].loc[df[busca] == x].iloc[0]
    else:
        return 'NaN'



In [3]:

Base = pd.read_csv(r'ENEM_TRATADO.csv', engine="python").dropna()
Base.head(5)

,Unnamed: 0.1,Unnamed: 0,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,CO_ESCOLA,RECURSO_ESPECIAL,Cor_Branca,Cor_Preta,Cor_Parda,Cor_Amarela,Cor_Indígena,ESC_PUBL,CAT_CH,CAT_CN,CAT_LC,CAT_MT,CAT_REDACAO,Estudo_Pai,Estudo_Mae,Ocupacao_Pai_1,Ocupacao_Pai_2,Ocupacao_Pai_3,Ocupacao_Pai_4,Ocupacao_Pai_5,Ocupacao_Mae_1,Ocupacao_Mae_2,Ocupacao_Mae_3,Ocupacao_Mae_4,Ocupacao_Mae_5,Pessoas_casa,Renda
0,2,2,18,1,1,15230023.0,0,1,0,0,0,0,0,3,3,3,3,3,0.000000,0.666667,1,0,0,0,0,0,0,0,1,0,0.105263,0.176471
1,7,7,26,1,0,15042626.0,0,0,0,1,0,0,1,2,2,1,2,2,0.166667,0.166667,0,1,0,0,0,0,1,0,0,0,0.157895,0.117647
2,16,16,17,1,1,15038378.0,0,0,0,1,0,0,1,2,2,2,2,2,0.666667,0.666667,0,1,0,0,0,0,1,0,0,0,0.421053,0.176471
3,24,24,18,1,1,15582949.0,0,0,0,1,0,0,1,2,2,2,2,2,0.666667,0.666667,0,1,0,0,0,0,1,0,0,0,0.105263,0.176471
4,30,30,17,0,1,15007375.0,0,0,0,1,0,0,1,3,2,3,3,3,0.500000,0.166667,1,0,0,0,0,1,0,0,0,0,0.105263,0.000000


# Trata os dados de Adequação de Formação Docente (AFD)

### Grupo 1 - Docentes com formação superior de licenciatura (ou bacharelado com complementação pedagógica) na mesma área da disciplina que leciona.

### Grupo 2 - Docentes com formação superior de bacharelado (sem complementação pedagógica) na mesma área da disciplina que leciona.

### Grupo 3 - Docentes com formação superior de licenciatura (ou bacharelado com complementação pedagógica) em área diferente daquela que leciona.

### Grupo 4 - Docentes com formação superior não considerada nas categorias anteriores.

### Grupo 5 - Docentes sem formação superior.

Os dados originais trazem a informação em termos de porcentagem de cada grupo.

A preparação leva em concideração apenas os dados relativos ao ensino médio.

Os dados selecionados são normalizados para a escala de 0 a 1.

Como há uma interdependência entre os valores dos grupos (a soma de todos os grupos é sempre 100) informar ao modelo os valores dos 5 grupos seria uma redundância. Decidiu-se eliminar o grupo 5 (menos frequênte).


In [4]:
#Carrega dados de AFD
DF_AFD = pd.read_csv(r'AFD_2019.csv',engine="python", sep=';',encoding= 'ISO-8859-1',decimal=',')

# Declara as colunas de interesse
Colunas = ['MED_CAT_1', 'MED_CAT_2', 'MED_CAT_3', 'MED_CAT_4']
# Guarda o código das entidades 
CodEnt = DF_AFD['CO_ENTIDADE']

# Remove colunas que não são as desejadas
remover = [coluna for coluna in DF_AFD.columns if coluna not in Colunas]
DF_AFD = DF_AFD.drop(columns = remover)

# Converte str "--" em int 0 
DF_AFD = DF_AFD.applymap(lambda x: 0 if (x=='--') else x)
DF_AFD = DF_AFD.astype(float).applymap(lambda x: x/100)

#Reincere o código da entidade no dataframe final
DF_AFD.insert(loc=0, column="CO_ENTIDADE", value =CodEnt)

In [5]:
DF_AFD

,CO_ENTIDADE,MED_CAT_1,MED_CAT_2,MED_CAT_3,MED_CAT_4
0,11022558,0.000,0.000,0.000,0.000
1,11024275,0.000,0.000,0.000,0.000
2,11024291,0.000,0.000,0.000,0.000
3,11024372,0.000,0.000,0.000,0.000
4,11024666,0.000,0.000,0.000,0.000
...,...,...,...,...,...
177709,53084020,0.000,0.000,0.000,0.000
177710,53084039,0.727,0.000,0.091,0.091
177711,53084055,1.000,0.000,0.000,0.000
177712,53085000,0.000,0.000,0.000,0.000


# Trata os dados de Alunos por Turma (ATU)
Média de alunos por turma.
Optou-se por tratar somente as turmas de terceiro ano do EM

In [6]:
#Carrega dados de ATU
DF_ATU = pd.read_csv(r'ATU_2019.csv',engine="python", sep=';',encoding= 'ISO-8859-1',decimal=',')

# Declara as colunas de interesse
Colunas = ['MED_03_CAT_0']
# Guarda o código das entidades 
CodEnt = DF_ATU['CO_ENTIDADE']

# Remove colunas que não são as desejadas
remover = [coluna for coluna in DF_ATU.columns if coluna not in Colunas]
DF_ATU = DF_ATU.drop(columns = remover)

# Converte str "--" em int 0 
DF_ATU = DF_ATU.applymap(lambda x: 0 if (x=='--') else x).astype(float)

Max = DF_ATU.iloc[:,0].max()

# Normaliza os valores entre 0 e 1
DF_ATU = DF_ATU.astype(float).applymap(lambda x: x/Max)

#Reincere o código da entidade no dataframe final
DF_ATU.insert(loc=0, column="CO_ENTIDADE", value =CodEnt)

In [7]:
DF_ATU

,CO_ENTIDADE,MED_03_CAT_0
0,11022558,0.000000
1,11024275,0.000000
2,11024291,0.000000
3,11024372,0.000000
4,11024666,0.000000
...,...,...
176381,53084020,0.000000
176382,53084039,0.020101
176383,53084055,0.150754
176384,53085000,0.000000


# HAD
## Tratamento dos dados de Horas Por Aula diárias



In [8]:
#Carrega dados de ATU
DF_HAD = pd.read_csv(r'HAD_2019.csv',engine="python", sep=';',encoding= 'ISO-8859-1',decimal=',')

# Declara as colunas de interesse
Colunas = ['MED_03_CAT_0']
# Guarda o código das entidades 
CodEnt = DF_HAD['CO_ENTIDADE']

# Remove colunas que não são as desejadas
remover = [coluna for coluna in DF_HAD.columns if coluna not in Colunas]
DF_HAD = DF_HAD.drop(columns = remover)


# Converte str "--" em int 0 
DF_HAD = DF_HAD.applymap(lambda x: 0 if (x=='--') else x).astype(float)

Max = DF_HAD.iloc[:,0].max()

# Normaliza os valores entre 0 e 1
DF_HAD = DF_HAD.astype(float).applymap(lambda x: x/Max)

#Reincere o código da entidade no dataframe final
DF_HAD.insert(loc=0, column="CO_ENTIDADE", value =CodEnt)

In [9]:
DF_HAD.describe()

,CO_ENTIDADE,MED_03_CAT_0
count,1.525090e+05,27303.000000
mean,3.156629e+07,0.322112
std,9.459077e+06,0.091163
min,1.100004e+07,0.018750
25%,2.509920e+07,0.268750
50%,3.134937e+07,0.300000
75%,3.547074e+07,0.331250
max,5.308601e+07,1.000000


In [10]:
%%time

Base['AFD_1'] = Base['CO_ESCOLA'].apply(lambda x: match(x, DF_AFD, 'CO_ENTIDADE', 'MED_CAT_1')).dropna().astype(float)
print('AFD_1')

Base['AFD_2'] = Base['CO_ESCOLA'].apply(lambda x: match(x, DF_AFD, 'CO_ENTIDADE', 'MED_CAT_2')).dropna().astype(float)
print('AFD_2')


Base['AFD_3'] = Base['CO_ESCOLA'].apply(lambda x: match(x, DF_AFD, 'CO_ENTIDADE', 'MED_CAT_3')).dropna().astype(float)
print('AFD_3')

Base['AFD_4'] = Base['CO_ESCOLA'].apply(lambda x: match(x, DF_AFD, 'CO_ENTIDADE', 'MED_CAT_4')).dropna().astype(float)
print('AFD_4')

Base['ATU'] = Base['CO_ESCOLA'].apply(lambda x: match(x, DF_ATU, 'CO_ENTIDADE', 'MED_03_CAT_0')).dropna().astype(float)
print('ATU')

Base['HAD'] = Base['CO_ESCOLA'].apply(lambda x: match(x, DF_HAD, 'CO_ENTIDADE', 'MED_03_CAT_0')).dropna().astype(float)
print('HAD')


AFD_1
AFD_2
AFD_3
AFD_4
ATU
HAD
CPU times: total: 1min 50s
Wall time: 1min 56s


In [11]:
Base = Base.drop(columns= ['CO_ESCOLA', 'Unnamed: 0'])

Base.to_csv('ENEM_MESCLADOS_teste.csv')